In [4]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

def process_kboat_data(year):
    def read_race_entry(year):
        entry = pd.read_csv(f'./crawlled_data/kboat_entries_{year}.csv')
        entry = entry.drop_duplicates()
        entry['Race_ID'] = entry.apply(lambda row: f"{row['연도']}_{row['회차']}_{row['일차']}_{row['경주번호']}", axis=1)
        columns = ['Race_ID'] + [col for col in entry.columns if col != 'Race_ID']
        entry = entry[columns].reset_index(drop=True)
        return entry

    def extract_first_player(df):
        del_list = ['①', '②', '③', '④', '⑤', '⑥']
        for item in del_list:
            df = df.replace(item, " ")
        return df[:5].strip()  # 첫 번째 3글자를 반환 (공백 포함)

    def read_race_result(year):
        result = pd.read_csv(f'./crawlled_data/kboat_result_{year}.csv')
        result = result.drop_duplicates()
        result['경주'] = result['경주'].replace('R', '', regex=True)
        result['경주'] = result['경주'].str.lstrip('0')
        result['Race_ID'] = result.apply(lambda row: f"{row['연도']}_{row['회차']}_{row['일차']}_{row['경주']}", axis=1)
        result = result[['Race_ID', '1위', '2위', '3위']].reset_index(drop=True)

        result["1위"] = result["1위"].apply(extract_first_player)
        result["2위"] = result["2위"].apply(extract_first_player)
        result["3위"] = result["3위"].apply(extract_first_player)
        return result

    # Entry와 Result 데이터 읽기
    entry = read_race_entry(year)
    result = read_race_result(year)

    # 두 DataFrame 병합
    merged_df = entry.merge(result[['Race_ID', '1위', '2위', '3위']], on='Race_ID', how='left')

    # rank 열을 계산
    conditions = [
        merged_df['선수명'] == merged_df['1위'],
        merged_df['선수명'] == merged_df['2위'],
        merged_df['선수명'] == merged_df['3위']
    ]
    choices = [1, 2, 3]
    merged_df['rank'] = np.select(conditions, choices, default=0)

    # 필요없는 열 제거
    merged_df = merged_df.drop(columns=['1위', '2위', '3위'])

    # Step 1: Race_ID별로 그룹화하고 각 그룹의 크기를 계산
    group_sizes = merged_df.groupby('Race_ID').size()

    # Step 2: 크기가 6이 아닌 Race_ID를 필터링
    invalid_race_ids = group_sizes[group_sizes != 6].index

    # Step 3: invalid_race_ids에 해당하는 행을 제거
    filtered_df = merged_df[~merged_df['Race_ID'].isin(invalid_race_ids)]

    # Step 4: rank 기준으로 3개가 아닌 Race_ID도 필터링
    rank_count = filtered_df.groupby('Race_ID')['rank'].apply(lambda x: (x != 0).sum())
    invalid_race_nos = rank_count[rank_count != 3].index
    final_df = filtered_df[~filtered_df['Race_ID'].isin(invalid_race_nos)]

    # 결과 DataFrame 반환
    final_df = final_df.reset_index(drop=True)
    return final_df



# year = 2016
# final_df = process_kboat_data(year)

In [5]:
# 학습 데이터

train = []

for year in range(2013, 2023+1):
    print(f"Processing year: {year}")
    final_df = process_kboat_data(year)  # 각 연도의 데이터를 처리
    train.append(final_df)  # 리스트에 추가


train_df = pd.concat(train, ignore_index=True)
# train_df.fillna('결측값', inplace=True)
train_df.to_csv(f'./data/train.csv', index=False, encoding='utf-8-sig')

Processing year: 2013
Processing year: 2014
Processing year: 2015
Processing year: 2016
Processing year: 2017
Processing year: 2018
Processing year: 2019
Processing year: 2020
Processing year: 2021
Processing year: 2022
Processing year: 2023


In [4]:
## 검증 데이터

test_df = process_kboat_data(2024)
test_df.to_csv(f'./data/test.csv', index=False, encoding='utf-8-sig')

In [1]:
## 테스트 데이터

import subprocess

def crawl_data_for_year(year):
    subprocess.run(["python", "crawlling_entry.py", str(year)])
    subprocess.run(["python", "crawlling_result.py", str(year)])
    print(f"Crawling completed for {year}.")

crawl_data_for_year(2024)

Crawling completed for 2024.


In [6]:
test_df = process_kboat_data(2024)
test_df

,Race_ID,연도,회차,일차,경주번호,번호,등급,기수,선수명,성별,...,모터_평균착순점,모터_연대율2,모터_연대율3,전탑승선수1,전탑승선수2,보트번호,보트_평균착순점,보트_연대율,특이사항,rank
0,2024_1_1_1,2024,1,1,1,1,B1,15,정세혁,남,...,4.98,29.7,47.9,박설희/536,이상문/466,99,5.50,39.1,NaN,0
1,2024_1_1_1,2024,1,1,1,2,B1,2,김기한,남,...,5.49,38.7,54.0,구본선/54,김창규/323,23,4.27,22.6,NaN,3
2,2024_1_1_1,2024,1,1,1,3,B1,11,기광서,남,...,5.35,35.7,52.3,한성근/35,한 진/636,93,5.39,37.3,NaN,2
3,2024_1_1_1,2024,1,1,1,4,B1,1,권명호,남,...,5.41,36.2,52.8,나종호/4643,정용진/511,67,5.60,37.8,NaN,0
4,2024_1_1_1,2024,1,1,1,5,A1,11,박진서,남,...,4.85,31.3,44.8,손제민/422,김태규/5252,78,5.14,33.9,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6097,2024_36_1_17,2024,36,1,17,2,B2,10,이경섭,남,...,4.79,34.5,44.8,조규태/561,김미연/36,81,4.38,33.3,NaN,3
6098,2024_36_1_17,2024,36,1,17,3,A1,13,김민준,남,...,7.46,61.5,76.9,주은석/1114,홍진수/135,58,5.58,38.7,NaN,1
6099,2024_36_1_17,2024,36,1,17,4,A2,7,배혜민,남,...,6.29,45.2,67.7,한성근/66,이승일/4224,74,4.78,34.4,NaN,0
6100,2024_36_1_17,2024,36,1,17,5,A1,8,이동준,남,...,4.41,25.0,37.5,정경호/633,조규태/4436,97,4.11,22.9,NaN,0


In [7]:
test_df.to_csv(f'./data/test.csv', index=False, encoding='utf-8-sig')